In [61]:
'''
DNN (tensorflow)による MNIST 分類
(メモ) dropout によって性能は上がる (91% → 94%) が，同時に early stopping 機構を組み込む必要あり。
(メモ) tensorflow は early stopping 機構はあるのかしら？ Keras には実装されているようだが。
'''
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
# sess = tf.InteractiveSession()
sess = tf.Session()

# モデルの作成
x = tf.placeholder(tf.float32, [None, 784])

'''
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
'''

'''
W = weight_variable([784, 10])
b = bias_variable([10])
y = tf.nn.softmax(tf.matmul(x, W) + b)
'''

enable_dropout = tf.placeholder_with_default(False, [], name="enable_dropout")
h1_keep_prob = tf.constant(0.5, dtype=tf.float32)

W1 = weight_variable([784, 362])
b1 = bias_variable([362])
h1 = tf.nn.relu(tf.matmul(x, W1) + b1)
h1_drop = tf.nn.dropout(h1, h1_keep_prob)

W2 = weight_variable([362, 10])
b2 = bias_variable([10])
y = tf.nn.softmax(tf.matmul(h1_drop, W2) + b2)

# 損失とオプティマイザーを定義
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
 
# 訓練
tf.initialize_all_variables().run()
for i in range(2700): # 元の値は 1000 である
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys, enable_dropout: True})

    if i % 10 == 0:
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("[{}] {}".format(i+1, accuracy.eval({x: mnist.test.images, y_: mnist.test.labels, enable_dropout: False})))

# 訓練モデルのテスト
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
[1] 0.16920000314712524
[11] 0.65829998254776
[21] 0.730400025844574
[31] 0.761900007724762
[41] 0.7968000173568726
[51] 0.8402000069618225
[61] 0.8048999905586243
[71] 0.8615999817848206
[81] 0.8105999827384949
[91] 0.8790000081062317
[101] 0.8859000205993652
[111] 0.8497999906539917
[121] 0.863099992275238
[131] 0.9035999774932861
[141] 0.8840000033378601
[151] 0.8830000162124634
[161] 0.8946999907493591
[171] 0.8932999968528748
[181] 0.8860999941825867
[191] 0.8956000208854675
[201] 0.8920000195503235
[211] 0.8984000086784363
[221] 0.8970999717712402
[231] 0.9097999930381775
[241] 0.8971999883651733
[251] 0.9070000052452087
[261] 0.9172000288963318
[271] 0.9161999821662903
[281] 0.9128000140190125
[291] 0.9180999994277954
[301] 0.920799970626831
[311] 0.9133999943733215
[321] 0.9203000068664551
[

In [57]:
'''
MLP(scikit-learn)による MNIST 分類
'''
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# mnistの手書き数字データをロード
# カレントディレクトリ（.）にない場合は、
# Webから自動的にダウンロードされる（時間がかかるので注意！）
# 70000サンプル、28x28ピクセル
X, y = sklearn.datasets.fetch_openml('mnist_784', version=1, return_X_y=True)

# ピクセルの値を0.0-1.0に正規化
X = X.astype(np.float64)
X /= X.max()

# 訓練データとテストデータに分解
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# 教師信号の数字を1-of-K表記に変換
#labels_train = LabelBinarizer().fit_transform(y_train)
#labels_test = LabelBinarizer().fit_transform(y_test)

# 多層パーセプトロンを構築
# mlp = MLPClassifier(max_iter=1000, early_stopping=True)
mlp = MLPClassifier((362, 181), max_iter=1000, early_stopping=True)

# 訓練データを用いてニューラルネットの重みを学習
mlp.fit(X_train, y_train)

# テストデータを用いて予測精度を計算
print(mlp.score(X_test, y_test))

y_pred = mlp.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
'''
predictions = []
for i in range(X_test.shape[0]):
    o = mlp.predict(X_test[i])
    predictions.append(np.argmax(o))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
'''

0.9818571428571429
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       678
           1       0.99      0.99      0.99       806
           2       0.98      0.99      0.98       686
           3       0.98      0.99      0.98       737
           4       0.99      0.98      0.98       671
           5       0.98      0.96      0.97       634
           6       0.98      0.99      0.99       671
           7       0.98      0.98      0.98       730
           8       0.99      0.97      0.98       691
           9       0.96      0.98      0.97       696

   micro avg       0.98      0.98      0.98      7000
   macro avg       0.98      0.98      0.98      7000
weighted avg       0.98      0.98      0.98      7000



'\npredictions = []\nfor i in range(X_test.shape[0]):\n    o = mlp.predict(X_test[i])\n    predictions.append(np.argmax(o))\nprint(confusion_matrix(y_test, predictions))\nprint(classification_report(y_test, predictions))\n'

In [102]:
'''
Kerasによる MNIST 分類
'''
import tensorflow.contrib.keras
from tensorflow.contrib.keras.api.keras.datasets import mnist
from tensorflow.contrib.keras.api.keras.models import Sequential
from tensorflow.contrib.keras.api.keras.layers import Dense, Dropout
from tensorflow.contrib.keras.api.keras.optimizers import RMSprop

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

batch_size = 128
num_classes = 10
epochs = 20

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2478 - acc: 0.9240 - val_loss: 0.1210 - val_acc: 0.9635
Epoch 2/20
60000/60000 [==============================] - 3s 42us/sample - loss: 0.1047 - acc: 0.9686 - val_loss: 0.0779 - val_acc: 0.9749
Epoch 3/20
60000/60000 [==============================] - 3s 43us/sample - loss: 0.0742 - acc: 0.9774 - val_loss: 0.0944 - val_acc: 0.9724
Epoch 4/20
60000/60000 [==============================] - 3s 43us/sample - loss: 0.0593 - acc: 0.9820 - val_loss: 0.0730 - val_acc: 0.9803
Epoch 5/20
60000/60000 [==============================] - 3s 43us/sample - loss: 0.0507 - acc: 0.9852 - val_loss: 0.0794 - val_acc: 0.9795
Epoch 6/20
60000/60000 [==============================] - 3s 43us/sample - loss: 0.0441 - acc: 0.9867 - val_loss: 0.0693 - val_acc: 0.9823
Epoch 7/20
60000/60000 [==============================] - 3s 43us/sample - loss: 0.0367 - acc: 0.9891 - val_loss: 0.